In [1]:
#爬取Agoda
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import openpyxl as op
import time
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException, StaleElementReferenceException

book1=op.Workbook()
sh1=book1['Sheet']
sh1.title="20220501"
#如果要重跑可以用的
# book2=op.load_workbook("Agoda_台北_0501.xlsx")
# sh2=book2.active

sh1.append(["資料來源","飯店名稱","地址","入住日期","房型","價錢","整體狀況及整潔度","設施與設備","舒適程度","位置","員工素質與服務"])

def find_link(item_list):
    
    for x in range(4): #滾動頁面，取得每一頁所有的飯店Links
        item=driver.find_elements(By.XPATH,'//a[@class="PropertyCard__Link"]')
        time.sleep(2)
        driver.find_element_by_xpath('//body').send_keys(Keys.END)#滾動到頁面最下方
        item=driver.find_elements(By.XPATH,'//a[@class="PropertyCard__Link"]')
    item_list.extend(item)
    print("讀取成功")
    time.sleep(5)
    
    for item_data in item_list: #點進Links取得飯店資訊
        time.sleep(10)
            
        try:
            driver.execute_script('arguments[0].click()',item_data)
            time.sleep(2)
            
        #因為有開新分頁需要切換視窗，-1是最新的視窗
        num = driver.window_handles
        driver.switch_to.window(num[-1])

        except StaleElementReferenceException:
            continue

        time.sleep(10)#要給時間讓網頁資料顯示
        
        #取飯店名稱
        try:
            item1=driver.find_element_by_css_selector('h1[class="HeaderCerebrum__Name"]')
            name=item1.get_attribute("innerText")
            print(name)
            
        except TimeoutException:
            driver.execute_script("window.stop();")#網頁停止loading
            continue

        #地址
        item2=driver.find_element_by_css_selector('span[class="Spanstyled__SpanStyled-sc-16tp9kb-0 gwICfd kite-js-Span HeaderCerebrum__Address"]')
        address=item2.get_attribute("innerText")

        #房型
        try:
            if driver.find_element_by_css_selector('span[class="MasterRoom__HotelName"]').is_displayed():
                item3=driver.find_element_by_css_selector('span[class="MasterRoom__HotelName"]')
                bed=item3.text
                print(bed)

            elif driver.find_element_by_css_selector('span[class="MasterRoom-headerTitle--text"]').is_displayed():
                item3=driver.find_element_by_css_selector('span[class="MasterRoom-headerTitle--text"]')
                bed=item3.text
                print(bed)

        except NoSuchElementException: #排除沒有空房的飯店
            #建議 from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException, StaleElementReferenceException
            driver.close()#關閉視窗
            driver.switch_to.window(num[0]) #返回原始視窗
            continue

        #價錢
        item4=driver.find_element_by_css_selector('span[class="Spanstyled__SpanStyled-sc-16tp9kb-0 gwICfd kite-js-Span pd-price PriceDisplay pd-color"]')
        price=item4.text
        if price.find(",")>0:
            price=price.replace(",","")
        print(price)

        #評價
        item5=driver.find_elements(By.CSS_SELECTOR,'div[class="Review-travelerGrade"]')
        score_list=[]
        for score in item5: 

            if score.get_attribute("innerText")[:-5]=="整體狀況及整潔度" or "設施與設備" or "客房舒適度" or "位置" or "服務" :
                comment2=score.get_attribute("innerText")[-3:]
                score_list.append(comment2)
            elif "整體狀況及整潔度" not in score.get_attribute("innerText") or "設施與設備" not in score.get_attribute("innerText") or "客房舒適度" not in score.get_attribute("innerText") or "位置" not in score.get_attribute("innerText") or "服務" not in score.get_attribute("innerText"):
                score_list.append("none")
 
        result=[]           
        result.extend(["Agoda",name,address_f,"2022-05-01",bed,price])
        result.extend(score_list)
        sh1.append(result)
        book1.save("Agoda_台北_0501.xlsx")
        time.sleep(3)

        driver.close() #關閉視窗
        driver.switch_to.window(num[0]) #返回查詢頁面
        time.sleep(5)
   
    #以上是取出一頁所有飯店資訊    
        
    driver.find_element_by_xpath('//body').send_keys(Keys.END)#滾動到查詢頁面最下面
    
    #按下一頁
    try:
        button1=driver.find_element(By.XPATH,'//button[@class="btn pagination2__next"]')
        button1.click()
        item_list.clear()
        find_link(item_list) #呼喚def繼續執行

    except NoSuchElementException:
        pass

    return item_list

desired_capabilities = DesiredCapabilities.CHROME  #調整selinium讀取方式
desired_capabilities["pageLoadStrategy"] = "none" #直接使用DOM樹，進行操作
driver=webdriver.Chrome("C:\\Users\\Lihome\\chromedriver.exe")
driver.get("https://www.agoda.com/zh-hk/search?city=4951&locale=zh-hk&ckuid=f97ea9a2-e509-4de5-a1aa-c728fb6593e7&prid=0&currency=TWD&correlationId=e70a84b0-6875-4311-a64d-a2cfb3df3650&pageTypeId=103&realLanguageId=7&languageId=7&origin=TW&cid=1844104&userId=f97ea9a2-e509-4de5-a1aa-c728fb6593e7&whitelabelid=1&loginLvl=0&storefrontId=3&currencyId=28&currencyCode=TWD&htmlLanguage=zh-hk&cultureInfoName=zh-hk&machineName=hk-acmweb-2007&trafficGroupId=1&sessionId=uq42ojyvxglel1twsrkeopil&trafficSubGroupId=84&aid=130589&useFullPageLogin=true&cttp=4&isRealUser=true&mode=production&checkIn=2022-05-01&checkOut=2022-05-02&rooms=1&adults=2&children=0&priceCur=TWD&los=1&textToSearch=%E5%8F%B0%E5%8C%97%E5%B8%82&productType=-1&travellerType=1&familyMode=off")

#點掉防疫諮詢的彈跳視窗
WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.CSS_SELECTOR,'button[class="Buttonstyled__ButtonStyled-sc-5gjk6l-0 kYHirW Box-sc-kv6pi1-0 hVPGaU"]'))).click()

#選擇住宿類型
driver.find_elements(By.CSS_SELECTOR,'button[class="btn PillDropdown__Button"]')[4].click()

#點飯店
driver.find_element(By.CSS_SELECTOR,'span[class="filter-item-info AccomdType-34 "]').click()

#點確認
try:
    button_1=driver.find_element(By.XPATH,'//*[@id="FilterBar"]/div[1]/div[5]/div/aside/footer/div/span/button[2]')
    driver.execute_script("arguments[0].click()",button_1) #排除按鈕被擋住的問題
    
except ElementClickInterceptedException:
     print("擋住了")
        
item_list=[]
find_link(item_list)

print("結束")


C:\Users\Lihome\AppData\Local\Temp/ipykernel_1756/4264039961.py:146: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome("C:\\Users\\Lihome\\chromedriver.exe")
C:\Users\Lihome\AppData\Local\Temp/ipykernel_1756/4264039961.py:29: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//body').send_keys(Keys.END)


讀取成功


C:\Users\Lihome\AppData\Local\Temp/ipykernel_1756/4264039961.py:52: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  item1=driver.find_element_by_css_selector('h1[class="HeaderCerebrum__Name"]')
C:\Users\Lihome\AppData\Local\Temp/ipykernel_1756/4264039961.py:62: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  item2=driver.find_element_by_css_selector('span[class="Spanstyled__SpanStyled-sc-16tp9kb-0 gwICfd kite-js-Span HeaderCerebrum__Address"]')
C:\Users\Lihome\AppData\Local\Temp/ipykernel_1756/4264039961.py:84: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  if driver.find_element_by_css_selector('span[class="MasterRoom__HotelName"]').is_displayed():
C:\Users\Lihome\AppData\Local\Temp/ipykernel_1756/4264039961.py:85: Dep

德立莊酒店 (Hotel Midtown Richardson)
雙人房 - 無窗 (Double No Window)
990
和苑三井花園飯店 台北忠孝 (MGH Mitsui Garden Hotel Taipei Zhongxiao)
普通房(兩床) (Moderate Twin)
3191
台北東旅 (Hotel East Taipei)
經典房(雙人床或兩床) (Classic Double or Twin)
2235
首都大飯店松山館 (Capital Hotel Songshan)
台北凱達大飯店 (Caesar Metro Taipei)
豪華雙床房 (Deluxe Twin)
2814
板橋凱撒大飯店 (Caesar Park Hotel Banqiao)
Elite客房(兩床) (Elite Twin Bed Room)
2684
Hotel PaPa Whale
雅緻雙人房 (Elegant Double Room)
1016
迪樂商旅(非防疫旅館) (The Dealer Hotel)
雙人房（21:00 之後入住） (Double Room (Check in after 21:00))
1184
格拉斯麗台北飯店 (Hotel Gracery Taipei)
荷里活房(兩床) (Hollywood Twin)
2167
台北時代寓所 (Hotel Resonance Taipei, Tapestry Collection by Hilton)
客房(兩床) (Twin Guest Room)
4300
德立莊酒店 (Hotel Midtown Richardson)
雙人房 - 無窗 (Double No Window)
990
台北新板希爾頓酒店 (Hilton Taipei Sinban)
客房(兩床) (Twin Guest Room)
3100
大倉久和大飯店 (The Okura Prestige Taipei Hotel)
尊貴荷里活房(兩床) (Prestige Hollywood Twin Room)
3883
薆悅酒店野柳渡假館 (Inhouse Hotel Yehliu)
高級房(兩床) (Superior Twin)
2056
翰品酒店 新莊 (Chateau de Chine Xinzhuang)
和苑三井花園飯

C:\Users\Lihome\AppData\Local\Temp/ipykernel_1756/4264039961.py:128: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//body').send_keys(Keys.END)


讀取成功
天閣酒店台北士林 (THE Tango Hotel Taipei Shilin)
園景雙人房 (Double Room with Garden View)
2712
晴美公寓酒店 (Jolley Hotel)
家庭套房 (Family Suite)
4242
悠趣旅店 (URTRIP Hotel)
標準雙人房（無窗） (Standard Double Room (No Window))
1185
鉑泊客旅店 (Poshpacker Hotel)
平價單人房 - 需共用浴室,無窗 (Budget Single Room with Shared Bathroom, without Window)
661
北投老爺酒店 (Hotel Royal Beitou)
高級房(雙人床) (Superior Room with 1 Double Bed)
6949
F Hotel 台北蘆洲館 (F Hotel Luzhou Taipei)
標準房(兩床) (Standard Twin)
1577
帝景飯店 (Lake Hotel)
湖景標準房 (Standard Lake View)
1933
新驛旅店西門捷運店 (Cityinn Hotel Plus Ximending Branch)
標準房 - 禁煙 (Standard - Non-Smoking)
1169
倆人旅店 (Hotel Double One)
甜蜜休息房(地庫) (Hotel Double One Basement Sweet Room)
4193
捷絲旅 - 西門町館 (Just Sleep Hotel Ximending)
豪華房 (Deluxe Room)
2176
沃客商旅正義館 (Walker Hotel ZhengYi)
經濟雙人／雙床房 (Economy Double or Twin Room)
1018
台北慕軒 (Madison Taipei Hotel)
經典房(特大床) (Classic King)
4554
新驛旅店台北車站二館 (CityInn Hotel Taipei Station Branch II)
標準房 (Standard Room)
1533
福格大飯店 (Hotel Regalees)
正 旅館 (Just Inn Taipei)
双层床 (Bunk Bed )

和昌商旅・淡水館 (Sunrise Business Hotel - Tamsui)
標準雙人或雙床間 (Standard Double or Twin Room)
1286
士林公寓 (Shihlin Service Apartment)
標準雙人房 (Standard Double)
1158
密都飯店 (Meadow Hotel Taipei)
經濟雙人間（無窗） (Economy Double Room (No Window))
1082
北海岸愛之船 (Ferry of Love Resort)
客房(2張大床/1樓) - 禁煙 (2 Queen Beds First Floor Non-Smoking)
3707
全家旅店 (Family Hotel Taipei)
標準雙人套房 (Standard Double Suite)
1393
文林居 (Family House)
客房(兩床) (Twin Room)
966
峻美精品商旅 (Beauty Hotels Taipei - Hotel Bnight)
經濟雙人房 (Economy Double Room)
1091
虹都商旅 (Hon Dow Hotel)
烏來驛站 (Courier Station)
尊爵雙人房 (Premier Double Room)
2292
板橋王旅館 (Banciaoking Hotel)
Standard Double Room
762
柯達大飯店永和店 (K Hotel - Yunghe)
Power Studio
2288
愛玩客旅店-西門町館 (I PLAY Inn)
標準雙人房 (Standard Double)
866
烏來淞呂溫泉會館 (WULAI SungLyu Spring Resort)
高級雙人房 (Superior Double Room)
2310
友統大飯店 (Yotong Hotel)
Standard Double Room
1039
皇家玫瑰旅館新生館 (Royal Rose Hotel Xinsheng)
標準房 (Standard Room)
1514
儷夏商旅 - 西門捷運館 (Muzik Hotel - Ximen Station Branch)
住宿安排房型- 限20:00後入住 (Run of House Room - Ch

小王子甜蜜會館 (Honey Prince Hotel)
標準雙人房 (Standard Double Room)
861
西門大飯店 - B館 (Ximen Hotel-B)
【限住2小時】日用雙人房 (Double Room - Day Use, Max 2 Hours Stay)
537
金弘旅店 (Goldenhome Hotel)
雙人房 (Double Room)
1089
貝斯特旅店 (Best Hotel Taipei)
豪華雙人房 (Deluxe Double)
1126
台北輕旅 SLEEP TAIPEI (Sleep Taipei Hostel & Hotel)
休息雙人房（8 小時） (Day-use Double Room (8 Hours))
1125
王朝大酒店 (Sunworld Dynasty Hotel)
Deluxe Double Room
2527
雅都旅社 (Ya Dou Hotel)
Deluxe Double Room
1109
甲山林湯旅 (JSL HOTEL)
設計雙人雙床房 (Design Twin Room)
2785
台北綠蒂飯店 (Taipei Lotty Hotel)
經典房(1張床) (Classic (One Bed Only))
1848
古山園旅社 (GU SHAN YUAN HOTEL)
標準雙人房 (Standard Double)
1039
璽愛旅店 (Say Love Hotel)
宿舍房的單人床 (Single Bed in Dormitory Room)
1195
結束
